# Preparing

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=5d92501d2c318973b4ca07591ff9cacb2f17be62c4383a4d10e6983d73f1ffbe
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName('assignment_dibimbing').setMaster('local[*]')
sc = SparkContext.getOrCreate(conf=conf)
sc

<SparkContext master=local[*] appName=assignment_dibimbing>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data Enginner/Day 6/diabetes.csv')

In [ ]:
data.info()

In [ ]:
data.head(
)

In [ ]:
data.sample(5)

# Explorasi Data

In [ ]:
numerical = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','Outcome']
data[numerical].describe()

In [ ]:
sns.countplot(data=data,
              x='Outcome',
              hue='Glucose')

Terlihat bahwa pada outcome (1) yaitu orang yg diabetes memiliki glucosa yang tinggi.

In [ ]:
correlation = data.corr()
sns.heatmap(correlation, annot=True, fmt=' .2f')

In [ ]:
sns.pairplot(data, hue='Outcome')

Terlihat bahwa pada Glucosa outcome(1) cenderung lebih ke kanan, jadi semakin tinggi glucosa memiliki kecenderungan mengalami diabetes.

# Spark Data Frame

In [ ]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("assignment_day6") \
    .getOrCreate()

# Read the CSV file
df = spark.read.csv("/content/drive/MyDrive/Data Enginner/Day 6/diabetes.csv", header=True, inferSchema=True)

# Print the schema
df.printSchema()

In [ ]:
df.show()

In [ ]:
filtered_df = df.filter(df['Glucose'] > 100)
filtered_df.show()

In [ ]:
filtered_df = df.where((df['Glucose'] > 100) & (df['Outcome'] == 1))
filtered_df.show()

In [ ]:
# Pilih kolom yang diinginkan
selected_df = filtered_df.select("Age", "Glucose", "Outcome")

# Tampilkan beberapa baris dari DataFrame yang telah dipilih kolomnya
selected_df.show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count
# Pilih kolom yang diperlukan
selected_df = filtered_df.select("Age", "Outcome")

# Filter hanya untuk yang terkena diabetes (Outcome == 1)
diabetes_df = selected_df.filter(col("Outcome") == 1)

# Group by Age dan hitung jumlah kasus diabetes untuk setiap umur
result_df = diabetes_df.groupBy("Age").agg(count("Outcome").alias("Count")).orderBy(col("Count").desc())

# Tampilkan beberapa baris dari DataFrame hasil
result_df.show()

Terlihat bahwa umur dibawah 30 tahun lebih banyak terkena diabetes

# Visualisasi

In [ ]:
import matplotlib.pyplot as plt
diabet_pd=(
    result_df
    .toPandas()
)

In [ ]:
diabet_pd.describe()

In [ ]:
diabet_top = diabet_pd.head(5)

In [ ]:
diabet_top.plot(x="Age",y="Count",kind="line")
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Count of Diabetes Cases by Age')
plt.grid(True)
plt.show()

Umur yang lebih tua tidak membuktikan sesorang memiliki gula yang tinggi dan terkena diabetes, terlihat bahwa umur dibawah 30tahun lebih banyak terkena diabetes.